## Generación de Dataset

In [43]:
import os
import pandas as pd
import pysrt

In [44]:
# Definir la ruta principal
ruta_principal = "/Users/adrianamartinez/Documents/NLP/project/datasets/"

In [45]:
# Crear listas para almacenar la información extraída
datos_peliculas = []

In [46]:
# Función para limpiar el nombre de la película
def limpiar_nombre_pelicula(nombre_archivo):
    subcadenas_a_eliminar = [".es", ".srt", "_spa", "_SPA", ".lat", "_lat", "LAT", "FRENCH", "Spanish", "-es"]
    for subcadena in subcadenas_a_eliminar:
        nombre_archivo = nombre_archivo.split(subcadena)[0]
    return nombre_archivo.strip().replace("."," ").replace("_"," ").replace("-"," ")

In [47]:
# Recorrer la estructura de directorios
for genero in os.listdir(ruta_principal):
    ruta_genero = os.path.join(ruta_principal, genero)
    if os.path.isdir(ruta_genero):
        for archivo in os.listdir(ruta_genero):
            if archivo.endswith('.srt'):
                ruta_srt = os.path.join(ruta_genero, archivo)
                nombre_pelicula = limpiar_nombre_pelicula(archivo)
                try:
                    subs = pysrt.open(ruta_srt, encoding='latin-1')
                    subtitulos = "\n".join([sub.text for sub in subs])
                    datos_peliculas.append({
                        'género': genero,
                        'nombre_pelicula': nombre_pelicula,
                        'subtitulos': subtitulos
                    })
                except Exception as e:
                    print(f"Error al procesar {ruta_srt}: {e}")

In [48]:
# Crear DataFrame a partir de los datos extraídos
df_peliculas = pd.DataFrame(datos_peliculas)

In [49]:
# Guardar el DataFrame en un archivo CSV para análisis posterior
#ruta_salida = os.path.expanduser('~/peliculas_procesadas.csv')

df_peliculas.to_csv('/Users/adrianamartinez/Documents/NLP/project/output/peliculas_procesadas.csv', index=False)

## Generación de sistema de recomendación

### Preprocesamiento de Texto

In [62]:
import os
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import pysrt
import joblib

In [54]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup

# Función para limpiar texto
def limpiar_texto(texto):
    # Eliminar etiquetas HTML
    texto = re.sub(r'<.*?>', '', texto)
    # Eliminar caracteres especiales y convertir a minúsculas
    texto = re.sub(r'\W', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = texto.lower()
    return texto

# Limpiar subtítulos en el DataFrame
df_peliculas['subtitulos_limpios'] = df_peliculas['subtitulos'].apply(limpiar_texto)

# Vectorizar subtítulos utilizando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_peliculas['subtitulos_limpios'])

In [55]:
df_peliculas

,género,nombre_pelicula,subtitulos,subtitulos_limpios
0,ROMANCE,"Peter Pan 2003 BDRip 23,976",<b>Todos los niños crecen</b>\n<b>excepto uno<...,todos los niños crecen excepto uno cenicienta ...
1,ROMANCE,Inferno 1999 720p BluRay H264 AAC RARBG,"En la primavera,\nla luna llena brilla...\npar...",en la primavera la luna llena brilla para el g...
2,ROMANCE,02x24 With This Ring Part I,- ¿Cómo se llama ese tipo?\n- Pinter.\n- No Pi...,cómo se llama ese tipo pinter no pinter el ch...
3,ROMANCE,Mad About You 1x20 The Spy Girl Who Loved Me...,- ¿Todavía te tienen en espera?\n- ¿Puedes cre...,todavía te tienen en espera puedes creerlo 10...
4,ROMANCE,Perseguido (1947),PERSEGUIDO\nTERRITORIO DE NUEVO\nMÉJICO A PRIN...,perseguido territorio de nuevo méjico a princi...
...,...,...,...,...
467,CRIME,Young Tiger 1973 1080p NF WEB DL DDP2 0 x264 E...,"- Tienes razÃ³n, es hermoso.\n- SÃ­.\nMira. Bo...",tienes razã³n es hermoso sã mira bonito â no ...
468,CRIME,La Mante S01 E01 Hardcoded Eng Subs Sno,"- Hola.\n- Hola, comisario Feracci, de la 36.\...",hola hola comisario feracci de la 36 agente n...
469,CRIME,Kurt Courtney (1999),"El 8 de Abril de\n1994, a las 8.40 am,\nfue ha...",el 8 de abril de 1994 a las 8 40 am fue hallad...
470,CRIME,"Sonatine 1993 DVDRip 23,976",Me parece bien\nque hagas negocios...\npero no...,me parece bien que hagas negocios pero no olvi...


In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, df_peliculas['género'], test_size=0.2, random_state=42)

In [63]:
# Entrenar un modelo de Naive Bayes
modelo_nb = MultinomialNB()
modelo_nb.fit(X_train, y_train)

MultinomialNB()

In [64]:
# Entrenar un modelo de Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [65]:
# Entrenar un modelo de SVM
modelo_svm = SVC(kernel='linear', C=1, random_state=42)
modelo_svm.fit(X_train, y_train)

SVC(C=1, kernel='linear', random_state=42)

In [68]:
# Evaluar el modelo Naive Bayes
y_pred_nb = modelo_nb.predict(X_test)
print("Modelo Naive Bayes")
print(f'Accuracy: {accuracy_score(y_test, y_pred_nb)}')
print(classification_report(y_test, y_pred_nb))



Modelo Naive Bayes
Accuracy: 0.17894736842105263
              precision    recall  f1-score   support

      ACTION       1.00      0.08      0.15        12
   ADVENTURE       0.00      0.00      0.00         9
      COMEDY       0.17      0.50      0.25         8
       CRIME       1.00      0.06      0.11        17
      HORROR       0.00      0.00      0.00        13
     MYSTERY       0.15      1.00      0.26         8
     ROMANCE       0.22      0.15      0.18        13
      SCI-FI       1.00      0.07      0.12        15

    accuracy                           0.18        95
   macro avg       0.44      0.23      0.14        95
weighted avg       0.52      0.18      0.13        95



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [69]:
# Evaluar el modelo Random Forest
y_pred_rf = modelo_rf.predict(X_test)
print("Modelo Random Forest")
print(f'Accuracy: {accuracy_score(y_test, y_pred_rf)}')
print(classification_report(y_test, y_pred_rf))


Modelo Random Forest
Accuracy: 0.21052631578947367
              precision    recall  f1-score   support

      ACTION       0.14      0.08      0.11        12
   ADVENTURE       0.00      0.00      0.00         9
      COMEDY       0.05      0.12      0.07         8
       CRIME       0.71      0.71      0.71        17
      HORROR       0.00      0.00      0.00        13
     MYSTERY       0.17      0.25      0.20         8
     ROMANCE       0.25      0.23      0.24        13
      SCI-FI       0.20      0.07      0.10        15

    accuracy                           0.21        95
   macro avg       0.19      0.18      0.18        95
weighted avg       0.23      0.21      0.21        95



In [70]:

# Evaluar el modelo SVM
y_pred_svm = modelo_svm.predict(X_test)
print("Modelo SVM")
print(f'Accuracy: {accuracy_score(y_test, y_pred_svm)}')
print(classification_report(y_test, y_pred_svm))

Modelo SVM
Accuracy: 0.35789473684210527
              precision    recall  f1-score   support

      ACTION       0.50      0.08      0.14        12
   ADVENTURE       0.16      0.33      0.21         9
      COMEDY       0.14      0.12      0.13         8
       CRIME       0.85      0.65      0.73        17
      HORROR       0.00      0.00      0.00        13
     MYSTERY       0.25      0.88      0.39         8
     ROMANCE       0.40      0.77      0.53        13
      SCI-FI       1.00      0.07      0.12        15

    accuracy                           0.36        95
   macro avg       0.41      0.36      0.28        95
weighted avg       0.48      0.36      0.31        95



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [71]:
# Calcular características estilométricas utilizando q-gramas
vectorizador_qgramas = CountVectorizer(analyzer='char', ngram_range=(1, 6))
X_qgramas = vectorizador_qgramas.fit_transform(df_peliculas['subtitulos_limpios'])

In [74]:
# Guardar los modelos y vectorizadores
joblib.dump(modelo_nb, '/Users/adrianamartinez/Documents/NLP/project/output/modelo_nb.pkl')
joblib.dump(modelo_rf, '/Users/adrianamartinez/Documents/NLP/project/output/modelo_rf.pkl')
joblib.dump(modelo_svm, '/Users/adrianamartinez/Documents/NLP/project/output/modelo_svm.pkl')
joblib.dump(vectorizer, '/Users/adrianamartinez/Documents/NLP/project/output/tfidf_vectorizer.pkl')
joblib.dump(vectorizador_qgramas, '/Users/adrianamartinez/Documents/NLP/project/output/qgramas_vectorizer.pkl')

['/Users/adrianamartinez/Documents/NLP/project/output/qgramas_vectorizer.pkl']

In [79]:
# Función para recomendar películas
def recomendar_peliculas(texto, df_peliculas, vectorizer, vectorizador_qgramas, modelo):
    texto_limpio = limpiar_texto(texto)
    texto_vectorizado = vectorizer.transform([texto_limpio])
    genero_predicho = modelo.predict(texto_vectorizado)[0]
    
    # Recomendación basada en género
    df_genero = df_peliculas[df_peliculas['género'] == genero_predicho]
    similitudes_genero = cosine_similarity(texto_vectorizado, vectorizer.transform(df_genero['subtitulos_limpios']))
    df_genero['similitud_genero'] = similitudes_genero[0]
    recomendacion_genero = df_genero.sort_values(by='similitud_genero', ascending=False).head(1)
    
    # Recomendación basada en estilo
    texto_qgramas = vectorizador_qgramas.transform([texto_limpio])
    similitudes_estilo = cosine_similarity(texto_qgramas, vectorizador_qgramas.transform(df_peliculas['subtitulos_limpios']))
    df_peliculas['similitud_estilo'] = similitudes_estilo[0]
    recomendacion_estilo = df_peliculas.sort_values(by='similitud_estilo', ascending=False).head(1)
    
    return recomendacion_genero[['nombre_pelicula', 'género', 'similitud_genero']], recomendacion_estilo[['nombre_pelicula', 'género', 'similitud_estilo']]


In [82]:
# Ejemplo de uso
texto_ejemplo = "Una emocionante aventura espacial con elementos de ciencia ficción."
recomendacion_genero, recomendacion_estilo = recomendar_peliculas(texto_ejemplo, df_peliculas, vectorizer, vectorizador_qgramas, modelo_nb)
print("Recomendación basada en género:")
print(recomendacion_genero)
print("\nRecomendación basada en estilo:")
print(recomendacion_estilo)

/var/folders/q_/x0m1ms_d5dv41d4l99_0gcl80000gn/T/ipykernel_14039/162612597.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genero['similitud_genero'] = similitudes_genero[0]


Recomendación basada en género:
                 nombre_pelicula   género  similitud_genero
150  The Originals S01E08 track4  MYSTERY          0.092762

Recomendación basada en estilo:
    nombre_pelicula  género  similitud_estilo
109            SEL9  SCI-FI          0.704635
